In [153]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm

In [154]:
dftrain=pd.read_csv('AAATrain.csv')

In [155]:
dftest=pd.read_csv('AAATest.csv')

In [156]:
model = sm.logit(formula='travel_destination_LAS ~ travel_total_bookings + travel_revenue_index + travel_tripvalue_index + \
                individual_education + individual_children_7_9 + days_between + Quartile_1 + Quartile_2 + Quartile_3', data=dftrain).fit()
print(model.summary())

         Current function value: 0.092191
         Iterations: 35


C:\Users\student\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                             Logit Regression Results                             
Dep. Variable:     travel_destination_LAS   No. Observations:                45800
Model:                              Logit   Df Residuals:                    45783
Method:                               MLE   Df Model:                           16
Date:                    Tue, 16 Jul 2024   Pseudo R-squ.:                 0.02033
Time:                            18:26:02   Log-Likelihood:                -4222.3
converged:                          False   LL-Null:                       -4310.0
Covariance Type:                nonrobust   LLR p-value:                 7.479e-29
                                                   coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Intercept                                       -4.6979      0.123    -38.065      0.000      -4.940      -4.4

In [157]:
Xtest = dftest[['travel_total_bookings', 'travel_num_cancelled_bookings', 'travel_revenue_index', 'travel_tripvalue_index', \
                'individual_education', 'individual_children_7_9', 'individual_children_13_18', \
                'days_between', 'Quartile_1', 'Quartile_2', 'Quartile_3', 'Quartile_4']]
ytest = dftest['travel_destination_LAS']

In [158]:
Vegas_yhat = model.predict(Xtest)

In [159]:
dftest['Vegas_yhat'] = Vegas_yhat

In [160]:
print(Vegas_yhat)

0        0.012150
1        0.029549
2        0.020546
3        0.014239
4        0.012483
           ...   
45795    0.028767
45796    0.012092
45797    0.040714
45798    0.013232
45799    0.017583
Length: 45800, dtype: float64


In [161]:
dftest = dftest.sort_values(by = 'Vegas_yhat', ascending = False)

In [162]:
dftest['Vegas_Quintile'] = pd.qcut(dftest['Vegas_yhat'], 5, labels=[i for i in range (5, 0, -1)])

In [163]:
print(dftest['Vegas_Quintile'])

3693     1
21550    1
20049    1
26530    1
10137    1
        ..
18654    5
28024    5
19674    5
25728    5
19374    5
Name: Vegas_Quintile, Length: 45800, dtype: category
Categories (5, int64): [5 < 4 < 3 < 2 < 1]


In [164]:
dftest['Vegas_Quintile'] = dftest['Vegas_Quintile'].astype('int')

In [165]:
Vegas_travelers = pd.crosstab(dftest['Vegas_Quintile'], dftest['travel_destination_LAS'], normalize='index')

In [166]:
print(Vegas_travelers)

travel_destination_LAS         0         1
Vegas_Quintile                            
1                       0.966128  0.033872
2                       0.976074  0.023926
3                       0.982143  0.017857
4                       0.984993  0.015007
5                       0.990076  0.009924


In [168]:
dftest.to_csv('VEGASfile.csv')